This notebook scrapes budget information from imdb.com for the top 1000 lifetime adjusted box office grosses, found on boxofficemojo.com. It exports the data as a list of tuples to 'budget_list.txt'.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [ ]:
page_number_url_list = list(range(0, 1000, 200))
page_number_url_list

In [ ]:
mainpage_url_list = []
mainpage_url = \
"https://www.boxofficemojo.com/chart/top_lifetime_gross_adjusted/?adjust_gross_to=2020&offset="

for i in page_number_url_list:
    mainpage_url_list.append(mainpage_url + str(i))

mainpage_url_list

In [ ]:
movie_id_list = []


for i in mainpage_url_list:
    mainpage = requests.get(i)
    soup = BeautifulSoup(mainpage.content, 'html.parser')    
    grandparent = soup.find('div', {'id': 'table'}).findAll('td', {'class': \
                            'a-text-left mojo-field-type-title'})
    for i in grandparent:
        x = i.find('a', {'class':'a-link-normal'}).get('href')
        each_movie_mainpage_url = str('https://www.boxofficemojo.com' + x)
        first_slice = str(each_movie_mainpage_url\
                        .split('https://www.boxofficemojo.com/title/', 1)[1])
        second_slice = str(first_slice.split('/?ref_', 1)[0])
        movie_id_list.append(second_slice)

movie_id_list

In [ ]:
movie_budget_list = []

for i in movie_id_list:
    imdb_url = 'https://www.imdb.com/title/'
    movie_url = str(imdb_url + i)
    mainpage = requests.get(movie_url)
    soup = BeautifulSoup(mainpage.content, 'html.parser') 
    try:
        grandparent = soup.find('div', {'id': 'titleDetails'}).find(text = 'Budget:')\
                            .parent.parent.get_text()
        first_slice = str(grandparent.split('Budget:$', 1)[1])
        second_slice = str(first_slice.split('\n', 1)[0])
        budget = second_slice.replace(',', '')
        print(i, budget)
        movie_budget_list.append((i, budget))
    except:
        pass

with open('budget_list.txt', 'w') as outfile:
    json.dump(movie_budget_list, outfile)

